# Veolia Data Science Internship &mdash; Technical Assignment

You are asked to work on the data of a wastewater treatment plant located in Southeast Asia.

First, I advise you to create a local environment using Python 3.11, and the packages you will find in `requirements.txt`.

With Conda, you can follow these steps (adapt for Python venv if needed):

```bash
conda create -n env_assignment python=3.11
conda activate env_assignment
pip install -r requirements.txt
```
___

## Goal

A colleague from the office of Veolia in Southeast Asia just sent you an email, asking for your help. 

She is in charge of a wastewater treatment plant, and would like you to explore the possibility of developping a tool that forecasts the volume of wastewater entering the plant every hour.

The objective here is to see if it is feasable, in which case you will showcase your proof of concept to this colleague.

You are provided with the following dataset, as is:

In [ ]:
import pandas as pd
import numpy as np
from utils.data import read_data

df = read_data()
df.head()

_ww_ means _wastewater_, if you had not guessed ;)

### Variables

| Variable Name                        | Type       | Unit         | Description                                                                                      |
|---------------------------------------|------------|--------------|--------------------------------------------------------------------------------------------------|
| wastewater volume                    | timeseries | m³       | **Target variable**. Total hourly volume of water entering the WWTP.                             |
| ww line 1                   | timeseries | m³       | Hourly volume of wastewater (pipe 1).                               |
| ww line 2                   | timeseries | m³       | Hourly volume of wastewater (pipe 2).                                     |
| ww line 3                  | timeseries | m³       | Hourly volume of wastewater (pipe 3).                    |
| ww line 4      | timeseries | m³           | Hourly volume of wastewater (pipe 4).                                    |
| rainfall                              | timeseries | mm      | Hourly rainfall amount measured on site.                                                                          |
| temperature_2m                        | timeseries | °C           | Air temperature at 2 meters above ground.                                                        |
| relative_humidity_2m                  | timeseries | %            | Relative humidity at 2 meters above ground.                                                      |
| precipitation                         | timeseries | mm           | Total precipitation (rain, showers, snow) in the preceding hour.                                 |
| rain                                  | timeseries | mm           | Only liquid precipitation (rain, showers) in the preceding hour.                                 |


#### Explanations

* The **wastewater volume** is the sum of four main inlet flows: ww lines 1, 2, 3 and 4. They are different type of wastewater lines, coming from different places.
* **rainfall** is measured on site by a sensor. 
* The other weather variables (four last lines of the table above) are gathered from a weather provider for the broad area of the plant, and are considered predictions. You may use those, and **only those**, for your wastewater forecast. **rainfall** or any water volume are _unknown_ for tduring your forecast would be a data leak.
___


## 🤖 Before You Start: A Note on AI Usage

I know many of you will ask an AI for help with this assignment. **That's perfectly fine** - AI is a powerful tool that you should learn to use efficiently. However, you should develop **critical thinking skills** about AI outputs.

### Some tasks to help you with that, and to see your reasoning:

1. If it is your way of working, **ask your favorite LLM** how to approach this assignment.
2. **Paste your initial conversation below** and then work as you would have done normally.
3. **At the end of your work**, identify what the AI initially got right, what it missed, and what might have been problematic (if anything) if you had strictly followed this path.


**Remember**: The goal is not to avoid AI, but to use it wisely. 

If you do not use AI, please mention it while still doing these steps for you to be able to review the initial AI output at the end of your assignment. If you have any comment, also feel free to elaborate on that below.


### 📝 Paste your AI conversation here:

*[Your AI query and response go here]*

### 🤔 Your critical analysis:

Throughout the project, I used ChatGPT as a complementary tool rather than a decision-maker. The core reasoning, modelling choices, and interpretation of results came from my own analysis, but ChatGPT was helpful for accelerating specific tasks such as clarifying certain Python behaviors, validating the intuition behind a methodological choice, or reformulating early ideas more clearly. I mainly relied on it the way one would consult a knowledgeable colleague—by asking targeted, technical questions rather than outsourcing the thinking. For example, at one point I used a prompt such as “Given a 7-day sliding window of learned representations, what are the pros and cons of using a simple linear head versus a tree-based model for 24-hour forecasting?” This type of interaction helped me refine my approach without influencing its direction, ensuring that the conceptual and modelling work remained my own.

---

## Your Work Starts Here

<div style="border-radius:10px; padding: 15px; background-color: rgb(36, 117, 179); font-size:130%; text-align:left">

### **Introduction & approach summary**

The first long task was to understand the dataset and the different relationships between the variables (very complex between correlation, autocorrelation, and multiple seasonal dependencies). 

My work was organized by going back and forth between tests and research, while also leaving time between sessions to take a step back from the problem and try to better understand its depth. 

After analyzing the data, I chose to test a simple initial model, which would give me some initial feedback on my analyses, then I tried to improve it using the results and other ideas about bias that came to me later. 

Finally, I wanted to spend some time looking for a model that would better match the complexity of the problem, namely a multivariate variable to predict (the sum of four variables with distinct temporal behaviors). While researching deep learning, I came across an article that addressed this type of multi-scale problem and also provided an example of implementation. In the end, this was the model that seemed to work best.

Below is a summary of the analyses carried out.


<a id="contents_tabel"></a>    
<div style="border-radius:10px; padding: 15px; background-color: rgb(36, 117, 179); font-size:130%; text-align:left">

<h2 align="left"><font>Table of Contents:</font></h2>
    
* [Part I - Charging the librairies](#librairies)
* [Part II - Initial Data Analysis](#initial_analysis) 
    - [II.1. Dataset Overview](#overview) 
    - [II.2. Summary Statistics](#statistics) 
    - [II.3. First vizualization of the data](#vizualization)
    - [II.4. Checking Correlation and Auto Correlation](#Correlation)
        - [Autocorrelation](#autocorrelation)
        - [Correlation](#correlation)
* [Part III - Data Cleaning & Transformation](#data_cleaning)
    - [III.1. Handling Missing Values](#missing_values)
    - [III.2. Handling Duplicates](#duplicates)
    - [III.3. Handling Outliers](#outliers)
* [Part IV - A First Simple Model, LightGBM](#LightGBM)
    - [IV.1. Feature Enginerring for LightGBM](#FE_LGBM)
        - [Part IV.1.A A New DataFrame](#new_dataframe) 
        - [Part IV.1.B Lags & rolling_means](#lags)
    - [Part IV.2 Implementing the model](#LGBM)
        - [Part IV.2.A Defining the target and feature selection](#target)
        - [Part IV.2.B Splitting the Data](#split)
        - [Part IV.1.C Finding the hyperparameters](#hyperparamters) 
        - [Part IV.1.D Model and Evaluation](#model) 
        - [Part IV.1.E Feature Importance](#feature)
* [PART V - Improving the LightGBM](#v2)
    - [Part V.1 Feature Enginerring for LightGBM v.2](#fe_2)
        - [Part V.1.A Target and lags](#lags2)
        - [Part V.1.B Autocorrelation based features](correlation2) 
        - [Part V.1.C Cyclical features](#cyclical) 
    - [Part V.2 Implementing the model](#model2)
        - [Part V.2.A Train/test split by shuffled 7-day blocks](#split2)
        - [Part V.2.B Splitting the Data](#kmeans_model)
        - [Part V.2.C Finding the hyperparameters](#evaluation)
        - [Part V.2.D Model and Evaluation](#evalutaion2)
* [PART VI - Testing a more advanced model using Deep Learning - Ts2Vec](#ts2vec)
    - [Part VI.1 Preparing the Data](#cluster_distributuion) 
    - [Part VI.2 Implementing the model](#evaluations_metrics)
        - [Part VI.2.A Training ts2vec](#profiling)
        - [Part VI.2.B Linear regression on the encoded train set](#radar_chart)
        - [Part VI.2.C Evaluting the model](#model3)
        - [Part VI.2.D Recreating the wastewater volume feature to plot the results](#plot)

<a id="librairies"></a>

# <p style="color:#E71B24; font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px; text-align:center;">PART I - Charging the libraries</p>
⬆️ [Tabel of Contents](#contents_tabel)


In [133]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from lightgbm import LGBMRegressor
import optuna
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from pandas.plotting import autocorrelation_plot
from ts2vec import TS2Vec
from sklearn.linear_model import LinearRegression
import torch

<p style="color:red"> xx mettre des commentaires expliquant à quoi servent chacune des librairies </p>

<a id="initial_analysis"></a>
# <p style="color:#E71B24; font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px; text-align:center;">PART II - First analysis on the data</p>

<a id="overview"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">  Part II.1 Dataset Overview</p>


In [ ]:
df.info()

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

**Inferences :** 

- The dataset contains 25727 entries, covering the 2018-2021 period and each entry corresponding to an hour of day during these 3 years. It is important to check if there are any gaps in the period.


- All the columns are of object type, except the rainfall one which is a float type. As they actually are all numbers, we will change these types to float, which will be easier to deal with.
Some columns (WW volume, WW line 4 and relative_humidity_2m) contain missing values which we will address later. 

In [ ]:
# Checking for eventual gaps in the time series
time_diffs = df.index.to_series().diff()
gaps = time_diffs[time_diffs > pd.Timedelta(hours=1)]

if gaps.empty:
    print("No gaps found in the time series.")

In [136]:
# Changing the dtypes of the columns to float type, taking care of some eventual commas as decimal separators
for col in df.columns:
    if col != "date":
        df[col] = (
            df[col]
            .astype(str)                  # converting to string first
            .str.replace(',', '.', regex=False)  # replacing commas with dots
            .astype(float)                # converting to float
        )

In [ ]:
df.info()

<a id="statistics"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part II.2 Some First Statistics on the Data

In [ ]:
df.describe()

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

#### Inferences :

**WW Volume :**  
- The mean ww volume entering the WWTP is 1535 with a large std of 955.5.  
- It has a wide range with a maximum at 14312, much higher than the 75% quartile, and which we will thus have to further investigate.  
- The 7 seven missing values will also be treated. 

**WW lines 1,2,3,4:**  
- Same inferences than the WW Volume's column.  
- The minimum values are not below 0, so there are no lower outliers, but the maximum values will need to be investigated.  

**Rainfall:**  
- Given the 25%, 50% and 75% quartiles, we can observe that the entries of this column are mainly zero.   
- The maximal value once again seems very big, but not physically impossible during a particularly stormy day.

**temperature_2m and relative_humidity_2m:**  
- These columns do not seem to have strange values (physically coherent), excepted for de 103% of relative humidity. It can however just a sensor's mistake due to noise as it is not that far from a coherent value. 
- The missing values of the relative humidity column will have to be addressed.

**rain and precipitation:**
- These columns seem to be close to identical, which is coherent considering they only differ on the snow counting. It will be further investigated wether it is useful to keep the two columns while applying the models. 
- We can also observe that the mean is very close to the rainfall column, which seems to indicate that the weather predications are rather accurate on average. 





<a id="vizualization"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part II.3 First vizualization of the data

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">In this section, we will take a first look at the data by testing different views (hourly, daily, weekly, montly). The goal is to have a first idea of the variations of the wastewater volume, its trends, and eventual seasonality. 

In [ ]:
# Visualize the wastewater volume over the entire time period

plt.figure(figsize=(14, 6))
plt.plot(df.index, df['wastewater volume'], label='Wastewater Volume (m³)', linewidth=1.5)
plt.title('Hourly variations in wastewater volume')
plt.xlabel('Time')
plt.ylabel('Volume (m³)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the wastewater volume over Nadays days for better clarity
i=1000
Ndays=5
plt.figure(figsize=(14, 6))
plt.plot(df.index[i:24*Ndays+i], df['wastewater volume'][i:24*Ndays+i], label='Wastewater Volume (m³)', linewidth=1.5)
plt.title('Hourly variations in wastewater volume')
plt.xlabel('Time')
plt.ylabel('Volume (m³)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

#### Inferences :

**Significant variability**  
Wastewater volume varies a little from hour to hour, with frequent peaks and periods of low activity.
Peaks can exceed 10,000 m³, while troughs often fall below 2,000 m³.

**Apparent seasonality**  
Repeating patterns are observed on regular cycles, suggesting daily or weekly seasonality.
Peaks seem to occur at similar times of day, probably linked to human activity (e.g., morning and evening).

**Long-term trend**  
There is no clear upward or downward trend over the entire period, but seasonal variations are noted

**Outliers**  
Some values are well above the average, which may indicate exceptional events (heavy rainfall, industrial incidents, or errors).


In [ ]:
df_mean = df.resample('d').mean()

plt.figure(figsize=(14, 6))
plt.plot(df_mean.index, df_mean['wastewater volume'], label='Wastewater Volume (m³)', linewidth=1.5)
plt.title('Daily changes in wastewater volume')
plt.xlabel('Time')
plt.ylabel('Volume (m³)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">
Daily variability

The average daily volume of wastewater generally varies between 1,000 m³ and 4,000 m³, with peaks of up to 7,000 m³.
Peaks are less frequent and smaller than in the hourly graph, but remain significant.

In [ ]:
df_mean = df.resample('W').mean()

plt.figure(figsize=(14, 6))
plt.plot(df_mean.index, df_mean['wastewater volume'], label='Wastewater Volume (m³)', linewidth=1.5)
plt.title('Weekly changes in wastewater volume')
plt.xlabel('Time')
plt.ylabel('Volume (m³)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">
Weekly variability  
 
Here we can see that the peaks are much less significant (maximum at 2600), which shows that large variations are very occasional (and almost entirely smoothed out over a week).

In [ ]:
df_mean = df.resample('M').mean()

plt.figure(figsize=(14, 6))
plt.plot(df_mean.index, df_mean['wastewater volume'], label='Wastewater Volume (m³)', linewidth=1.5)
plt.title('Monthly changes in wastewater volume')
plt.xlabel('Time')
plt.ylabel('Volume (m³)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">There appears to be annual seasonality, with recurring peaks at certain times of the year.
For example, peaks are observed in summer (June-September) and lows in winter (December-January). 

In [ ]:
variables = ['relative_humidity_2m', 'rain', 'temperature_2m']
titles = [
    'Relative Humidity %)',
    'Rainfall (mm/h)',
    'Air Temperature at 2m (°C)'
]

fig, axes = plt.subplots(len(variables), 1, figsize=(12, 6), sharex=True)

for i, var in enumerate(variables):
    axes[i].plot(df.index, df[var], linewidth=1.2, color='tab:blue')
    axes[i].set_title(titles[i], fontsize=11)
    axes[i].grid(True)
    axes[i].set_ylabel(var)

axes[-1].set_xlabel('Time')
plt.tight_layout()
plt.show()


<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">
Here we can see that the three variables have variations that appear to be seasonal, which was excepted for weather features

Their peaks also appear to be coordinated, which could indicate a correlation between them, which then again seems logical

<a id="Correlation"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part II.4 Checking Correlation and Auto Correlation

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

In this setion we continue to investigate the data set. We take a better look at the temporal dependencies within our dataset by examining the Autocorrelation Function (ACF) and the Partial Autocorrelation Function (PACF).


- **ATF :** measures how a time-series variable is correlated with its own past values across different lags. It helps reveal overall patterns such as persistence or repeating cycles.

- **PACF :** isolates the direct relationship between a variable and its past values at a specific lag, removing the effects of the intermediate lags. It is especially useful for identifying the appropriate number of autoregressive terms in time-series models.

 These tools help us identify patterns such as persistence, seasonality, and lagged relationships, which are essential for understanding the underlying dynamics of each variable and for guiding subsequent time-series modeling steps.

<a id="autocorrelation"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> II.4.A Autocorrelation

In [ ]:
variables = df.columns.tolist()
n_vars = len(variables)
fig, axes = plt.subplots(n_vars, 2, figsize=(12, n_vars * 3))  # 3 inches height per variable

for i, var in enumerate(variables):
    plot_acf(df[var].dropna(), ax=axes[i, 0], lags=48, title=f"ACF - {var}")
    plot_pacf(df[var].dropna(), ax=axes[i, 1], lags=48, title=f"PACF - {var}")

plt.tight_layout()
plt.show()


<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

**ACF**

- The ww lines 2-3 see their ACF decrease slightly and then fluctuate. This therefore shows cyclical autocorrelation, with high peaks occuring at lag 12,24 and 48, while low peaks occur at lags 6,18,30 etc
- The rain, rainfall, precipation and ww line have a slowly deacreasing ACF, suggesting that they are influenced by the 10-15 values coming before but much 
less by the further ones. 

**PCF**

- All the variables have a first (and sometimes second) big peak. 
- The values of the other lags are close to zero, indicating that short-term dependence is mainly captured by the first lag.


<a id="correlation"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> II.4.B Autocorrelation

In [ ]:
#Chacking correlaion

correlation_matrix = df.corr()

plt.figure(figsize=(8,6))  # taille de la figure
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, fmt=".2f")
plt.title("Matrice de Corrélation")
plt.show()


<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

#### Inferences 

**Wastewater Volume:**

Strong correlations with the four ww lines 1 as expected as it is thei sum.
Positive correlation with rainfall (0.58), suggesting that rainfall impacts wastewater volume the most of the weather variables.



**Wastewater Lines:**

ww line 1 and ww line 2 are highly correlated (0.60), indicating a potential shared influence or similar behavior.
ww line 3 and ww line 4 also show moderate correlations with other wastewater lines.



**Meteorological Variables:**

Rainfall shows a moderate positive correlation with wastewater volume (0.58) and various wastewater lines.
Temperature_2m and relative_humidity_2m have low or negligible correlations with wastewater volume, indicating a weaker influence.
Rain and precipitation are perfectly correlated (1.00), as expected, since they almost represent the same thing. 


It could be interesting to see if other correlations appear with a daily average, as some variables seem to have daily autocorrelations (as seen just before).

In [ ]:
#checking daily correlation

daily_df = df.resample('D').mean()
daily_correlation_matrix = daily_df.corr()

plt.figure(figsize=(8,6))  # taille de la figure
sns.heatmap(daily_correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, fmt=".2f")
plt.title("Daily average correlation matrix")
plt.show()

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

#### Inferences


**Wastewater Volume:**

Extremely high positive correlation with ww line 1 (0.99) and ww line 2 (0.93), indicating that these lines are primary contributors to the overall wastewater volume.
Strong positive correlation with ww line 4 (0.86) and ww line 3 (0.82).
High positive correlation with rainfall (0.96), suggesting a significant impact of rainfall on wastewater volume.



**Wastewater Lines:**

ww line 1 and ww line 2 are highly correlated (0.89), indicating similar behavior or shared influences.
ww line 3 and ww line 4 also show strong correlations with other wastewater lines, though slightly less than ww line 1 and ww line 2.



**Meteorological Variables:**

Rainfall shows a very high positive correlation with wastewater volume (0.96) and strong correlations with various wastewater lines, indicating its significant influence.
Rain and precipitation are perfectly correlated (1.00), as they likely represent similar measurements.
Temperature_2m and relative_humidity_2m have low correlations with wastewater volume, indicating a weaker influence.

<a id="data_cleaning"></a>
# <p style="color:#E71B24; font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px; text-align:center;">PART III - Data Cleaning & Transformation</p>

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

This step encompasses a comprehensive cleaning and transformation process to refine the dataset. It includes addressing missing values, eliminating duplicate entries, correcting anomalies, and other necessary adjustments to prepare the data for modeling.

<a id="missing_values"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part III.1 Handling Missing Values 

In [ ]:
#Visualising the missing values 
df[df.isna().any(axis=1)]

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

- We can see that the rows where the wastewater volume is missing are the same as the ones where the data on the ww line 4 is missing too. We thus can't just sum or subtract the other values to get the one missing. We can see however that the missing values don't follow each other. 

- The missing values in the humidity column are also isolated. 

Depending on the model used in the end, even if the missing values only represent 0,0003% of the dataset, deleting them could confuse the model. As they don't follow each other, and because we saw in the previous analyses that the humidity, the ww volume and the ww4 are very influenced by the values just before, a **linear interpolation** seems the safest option.
Since we can obtain the ww volume based on the four other ww, we are only going to do the linear interpolation on the humidty and the ww4.

In [ ]:
#linear interpolation on the humidity and the ww4 columns to fill the missing values

df['relative_humidity_2m'] = df['relative_humidity_2m'].interpolate(method='time')
df['ww line 4'] = df['ww line 4'].interpolate(method='time')

df['wastewater volume'] = df['wastewater volume'].fillna(df[['ww line 1','ww line 2','ww line 3','ww line 4']].sum(axis=1))

df[df.isna().any(axis=1)]

<a id="duplicates"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part III.2 Handling Duplicates 

In [ ]:
#Checking if there are any duplicated rows
duplicates = df[df.duplicated(keep=False)]  # keep=False to see all the duplicated rows
if duplicates.empty:
    print("No duplicated rows found.")


In [ ]:
# Checking if there are any duplicated timestamps
duplicated_timestamps = df.index[df.index.duplicated(keep=False)]
if duplicated_timestamps.empty:
    print("No duplicated timestamps found.")

<a id="outliers"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part III.3 Handling Outliers

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

The strategy here is to examine whether the abnormal values observed in a given variable are correlated with extreme values in the other columns.
- If such correlations exist, the anomalies are likely meaningful patterns rather than errors, and may carry useful information for the model. We will not try to adress them in this section because the sensitivity to outliers depends on the model, and thy should thus be adressed accordingly. 
- If no correlation is found, the values are more likely to be genuine errors, in which case further inspection is recommended.

In [ ]:
df[df['wastewater volume'] > 10000]

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

We can see that every row here has relatively high values for the weather variables, except for the fifth row. This row actually has a bigger issue : the wastewater volume is not the sum of the other four ww lines. Let's find out if there are others.

In [ ]:
# Explicit column list
cols = ['ww line 1', 'ww line 2', 'ww line 3', 'ww line 4']
total_col = 'wastewater volume'

# Calculate the sum of the components
components_sum = df[cols].sum(axis=1)

# Boolean mask where the total is NOT equal to the sum of components
mask = ~np.isclose(df[total_col], components_sum, atol=1e-1, rtol=1e-1)

# Extract only the inconsistent rows
mismatched_rows = df.loc[mask, df.columns].copy()

# Add a difference column for clarity
mismatched_rows['diff'] = df.loc[mask, total_col] - components_sum[mask]

# Display results
print(f"Number of inconsistent rows: {mask.sum()}")
display(mismatched_rows.head(5))

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">Only two rows have this issue, but to know what to fix, we need to take a look at the rows coming just before and after each of them.

In [ ]:
#Timestamp cible
timestamp_cible = pd.to_datetime("2019-06-04 03:00:00")
n = 5  # Nombre d'heures avant/après

# Trouver l'index du timestamp cible
index_cible = df.index.get_loc(timestamp_cible)

# Sélectionner les lignes
start = max(0, index_cible - n)
end = min(len(df), index_cible + n + 1)
df_selection = df.iloc[start:end]

df_selection

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

The outlier appears to be the wastewater volume, so it will be replaced by the sum of the other four values. Let's now take a look at the other problematic row.

In [ ]:
#Timestamp cible
timestamp_cible = pd.to_datetime("2019-07-07 14:00:00")
n = 5  # Nombre d'heures avant/après

# Trouver l'index du timestamp cible
index_cible = df.index.get_loc(timestamp_cible)

# Sélectionner les lignes
start = max(0, index_cible - n)
end = min(len(df), index_cible + n + 1)
df_selection = df.iloc[start:end]

df_selection

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">
Here, it is the value of ww line 4 that seems out of bound. Once again, we will use the other 4 to deal with the problem. 

In [156]:
# changing the wastewater volume and the ww line 4 values at the identified timestamps
df.loc["2019-06-04 03:00:00", "wastewater volume"] = df.loc["2019-06-04 03:00:00", cols].sum() 
df.loc["2019-07-07 14:00:00", "ww line 4"] = df.loc["2019-07-07 14:00:00","wastewater volume"] - df.loc["2019-07-07 14:00:00", ["ww line 1", "ww line 2", "ww line 3"]].sum() 

In [ ]:
# Calculate the sum of the components
components_sum = df[cols].sum(axis=1)

# Boolean mask where the total is NOT equal to the sum of components
mask = ~np.isclose(df[total_col], components_sum, atol=1e-1, rtol=1e-1)

# Extract only the inconsistent rows
mismatched_rows = df.loc[mask, df.columns].copy()

# Add a difference column for clarity
mismatched_rows['diff'] = df.loc[mask, total_col] - components_sum[mask]

# Display results
print(f"Number of inconsistent rows: {mask.sum()}")

<a id="LightGBM"></a>
# <p style="color:#E71B24; font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px; text-align:center;">PART IV - A First Simple Model, LightGBM</p>

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

To begin our predictive analysis of wastewater volume, we are going to useLightGBM (Light Gradient Boosting Machine). There are several reasons for this choice  :


**Simplicity and Speed**: LightGBM is an efficient and fast boosting algorithm, ideal for an initial exploratory model. It allows us to quickly obtain basic results without requiring complex configuration.  

**Data Management**: It is capable of handling tabular data with exogenous variables (here the meteorological data), which is relevant for our dataset.  

**Robustness**: LightGBM belongs to the decision tree family so it is robust to outliers and can capture nonlinear relationships, which is useful for time series with significant variations.

<a id="FE_LGBM"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part IV.1 Feature Enginerring for LightGBM

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

LightGBM is supposed to be robust to outliers, so we are not going to adress them for this first model.   
With all that we have seen before, every variable in the dataset has a strong temporal dependancy, so lags are going to be key. We are going to build new variables to best represent these previous observations. 
As the number of variables is not that important for this type of model (at least before 100 variables), we are going to keep all of them for the first try.

For every new model, I am going to create a new DataFrame dedicated to it, to keep in memory the old one and adapt the new one best to the model. 

<a id="new_dataframe"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part IV.1.A A New DataFrame

In [ ]:
LightGBM_df = df.copy()

#changing the name for better handling
LightGBM_df = LightGBM_df.rename(columns={
    'wastewater volume': 'ww0',
    'ww line 1': 'ww1',
    'ww line 2': 'ww2',
    'ww line 3': 'ww3',
    'ww line 4': 'ww4',
    'rainfall': 'rainfall_real',
    'rain': 'rain_forecast',
    'precipitation': 'precipitation_forecast',
    'relative_humidity_2m': 'humidity',
    'temperature_2m': 'temperature'
})

LightGBM_df = LightGBM_df.sort_index()  # assure chronological order


LightGBM_df

<a id="lags"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part IV.1.B Lags & rolling_means

In [159]:
# the last 24 values of ww0 are not know when prediciting batches of 24 hours, so we will remove them
LightGBM_df['rainfall_real_lag24'] = LightGBM_df['rainfall_real'].shift(24)

# Creating lag features for ww0 to ww4 from lag 24 to lag 48
for lag in range(24, 49):
    LightGBM_df[f'ww0_lag{lag}'] = LightGBM_df['ww0'].shift(lag)
    LightGBM_df[f'ww1_lag{lag}'] = LightGBM_df['ww1'].shift(lag)
    LightGBM_df[f'ww2_lag{lag}'] = LightGBM_df['ww2'].shift(lag)
    LightGBM_df[f'ww3_lag{lag}'] = LightGBM_df['ww3'].shift(lag)
    LightGBM_df[f'ww4_lag{lag}'] = LightGBM_df['ww4'].shift(lag)


In [160]:
#As observed in the daily correlation matrix, some variables have a significant correlation with the wastewater volume on a daily basis.

# Rolling means (6h, 12h, 24h)
LightGBM_df['ww0_roll6'] = LightGBM_df['ww0_lag24'].rolling(window=6).mean()
LightGBM_df['ww0_roll12'] = LightGBM_df['ww0_lag24'].rolling(window=12).mean()
LightGBM_df['ww0_roll24'] = LightGBM_df['ww0_lag24'].rolling(window=24).mean()


In [ ]:
LightGBM_df = LightGBM_df.dropna()

LightGBM_df

<a id="LGBM"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part IV.2 Implementing the model

<a id="target"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part IV.2.A Defining the target and feature selection

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">
We are here only focusing on the prediction of ww0, as the sum of the four others, and not of each of the ww lines separatly. 

In [162]:
target = 'ww0'

# we are not using the www lines as features directly, only their lags (otherwise the model is cheating)
features = [
    # Lags des ww lines et de ww0
    *[f'ww0_lag{i}' for i in range(25, 49)],
    *[f'ww1_lag{i}' for i in range(25, 49)],
    *[f'ww2_lag{i}' for i in range(25, 49)],
    *[f'ww3_lag{i}' for i in range(25, 49)],
    *[f'ww4_lag{i}' for i in range(25, 49)],
    # Rolling
    'ww0_roll6', 'ww0_roll12', 'ww0_roll24',
    # Météo
    'rainfall_real_lag24', 'rain_forecast', 'precipitation_forecast',
    'humidity', 'temperature'
]

X = LightGBM_df[features]
y = LightGBM_df[target]


<a id="split"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part IV.2.B Splitting the Data

In [163]:
split_index = int(len(LightGBM_df) * 0.75)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]


<a id="hyperparameters"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part IV.1.C Finding the hyperparameters

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

To enhance the performance of our LightGBM model, we utilize Optuna, an automated hyperparameter optimization framework. 

We define a search space for key LightGBM hyperparameters, such as learning_rate, num_leaves, max_depth, and n_estimators. Optuna then searches for the combination of hyperparameters that minimizes the validation error, leading to an optimized LightGBM model.

In [ ]:
def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'random_state': 42
    }

    model = LGBMRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

study = optuna.create_study(direction='minimize')
optuna.logging.set_verbosity(optuna.logging.ERROR)
study.optimize(objective, n_trials=50)
print(study.best_params)

<a id="model"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part IV.1.D Model and Evaluation

In [ ]:
#Avec Optina
model = LGBMRegressor(**study.best_params, random_state=42)

model.fit(X_train, y_train)

In [ ]:

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE = {mae:.2f}")
print(f"MSE = {mse:.2f}")
print(f"R²  = {r2:.3f}")



In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test.index[:500], y_test[:500], label='True', alpha=0.8)
plt.plot(y_test.index[:500], y_pred[:500], label='Predicted', alpha=0.8)
plt.title("LightGBM forecast — 24h ahead (first 500h of test set)")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test.index[500:1000], y_test[500:1000], label='True', alpha=0.8)
plt.plot(y_test.index[500:1000], y_pred[500:1000], label='Predicted', alpha=0.8)
plt.title("LightGBM forecast — 24h ahead (following 500h of test set)")
plt.legend()
plt.show()

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

We can see that the model learned very successfully the most common values (small oscillations around the mean value). 

However it did not at all predict the peaks, which are likely to be the ones really important to predict.

To find areas for improvement, we will look at feature importance.

<a id="feature"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part IV.1.E Feature Importance

In [ ]:
importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
plt.figure(figsize=(8,8))
importances.head(50).plot(kind='barh')
plt.title("Top 50 Feature Importances — LightGBM")
plt.show()


<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

To improve the performance and efficiency on the **LightGBM model**, we will be implementing several key optimizations and feature engineering strategies : 

**Focus on Important Features and Reduction of Lags**:  
- We will prioritize the use of **features identified as most important** from the feature importance analysis. This ensures that we focus on the variables that have the highest impact on predicting wastewater volume.
- Only few lags are very important (as shown in the feature importance analysis), so to simplify the model and reduce computational complexity, we will be using **fewer lags**. This helps in making the model lighter and faster while retaining its predictive power.

**Enhanced Feature Engineering**:
- We will have **additional feature engineering** on key meteorological variables such as **rainfall, rain forecast, humidity, and temperature**. (which are shown to be important).

**Incorporation of Autocorrelation Data**:
- We will also add **autocorrelation features**, which we did not really used in the previous version, to better capture the temporal dependencies in the data. These features help the model better understand the patterns and trends in the time series data, leading to more accurate predictions

By combining hyperparameter optimization with thoughtful feature engineering, there is good chance to believe the next version will succed better.

<a id="v2"></a>
# <p style="color:#E71B24; font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px; text-align:center;">PART V - Improving the LightGBM</p>

<a id="FE_LGBM"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part V.1 Feature Enginerring for LightGBM v.2

In [ ]:
LightGBM_v2_df = df.copy()
LightGBM_v2_df

<a id="model"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part V.1.A Target and lags

In [171]:
target_col = "wastewater volume"

#Shift ww to prevent data leakage
LightGBM_v2_df["target_col_24"] = LightGBM_v2_df[target_col].shift(24)
LightGBM_v2_df["ww1_24"] = LightGBM_v2_df["ww line 1"].shift(24)
LightGBM_v2_df["ww2_24"] = LightGBM_v2_df["ww line 2"].shift(24)
LightGBM_v2_df["ww3_24"] = LightGBM_v2_df["ww line 3"].shift(24)
LightGBM_v2_df["ww4_24"] = LightGBM_v2_df["ww line 4"].shift(24)

# Create lag features (we keep only the most relevant lags based on ACF/PACF and feature importance analysis)
for lag in [1, 2, 3, 6, 12, 18, 24, 36, 48]:
    LightGBM_v2_df[f"ww0_lag{lag}"] = LightGBM_v2_df["target_col_24"].shift(lag)
    LightGBM_v2_df[f"ww1_lag{lag}"] = LightGBM_v2_df["ww1_24"].shift(lag)
    LightGBM_v2_df[f"ww2_lag{lag}"] = LightGBM_v2_df["ww2_24"].shift(lag)
    LightGBM_v2_df[f"ww3_lag{lag}"] = LightGBM_v2_df["ww3_24"].shift(lag)
    LightGBM_v2_df[f"ww4_lag{lag}"] = LightGBM_v2_df["ww4_24"].shift(lag)

# Rolling window averages to capture recent trends
for window in [6, 12, 24]:
    LightGBM_v2_df[f"ww0_roll{window}"] = (
        LightGBM_v2_df["target_col_24"].rolling(window=window).mean()
    )
    
# Rolling window averages for ww lines
for window in [6, 12, 24]:
    LightGBM_v2_df[f"humidity_roll{window}"] = (
        LightGBM_v2_df["relative_humidity_2m"].rolling(window=window).mean()
    )


In [172]:
# Shift rainfall to prevent data leakage
LightGBM_v2_df["rainfall_lag24"] = LightGBM_v2_df["rainfall"].shift(24)
LightGBM_v2_df = LightGBM_v2_df.drop(columns=["rainfall"])

# Cumulative rainfall over different periods   
LightGBM_v2_df['rain_cum_6h']  = LightGBM_v2_df['rainfall_lag24'].rolling(6).sum()
LightGBM_v2_df['rain_cum_12h'] = LightGBM_v2_df['rainfall_lag24'].rolling(12).sum()
LightGBM_v2_df['rain_cum_24h'] = LightGBM_v2_df['rainfall_lag24'].rolling(24).sum()

# Cumulative rain over different periods
LightGBM_v2_df['rain_cum_6h']  = LightGBM_v2_df['rain'].rolling(6).sum()
LightGBM_v2_df['rain_cum_12h'] = LightGBM_v2_df['rain'].rolling(12).sum()
LightGBM_v2_df['rain_cum_24h'] = LightGBM_v2_df['rain'].rolling(24).sum()

<a id="model"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part V.1.B Autocorrelation based features

In [173]:
# Difference compared to previous day
LightGBM_v2_df["ww0_diff24"] = (
    LightGBM_v2_df["target_col_24"] - LightGBM_v2_df["target_col_24"].shift(24)
)

# Ratio to previous day (stability / abnormal variation)
LightGBM_v2_df["ww0_ratio24"] = (
    LightGBM_v2_df["target_col_24"] / (LightGBM_v2_df["target_col_24"].shift(24) + 1e-6)
)

# Local variation (standard deviation over 12 hours)
LightGBM_v2_df["ww0_std12"] = (
    LightGBM_v2_df["target_col_24"].rolling(window=12).std()
)

# 5. Drop rows with missing values (caused by shifts)
LightGBM_v2_df = LightGBM_v2_df.dropna()

<a id="model"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part V.1.C temporal / cyclical features

In [ ]:
# Cyclical encoding for hour and day of week
LightGBM_v2_df["hour_sin"] = np.sin(2 * np.pi * LightGBM_v2_df["hour"] / 24)
LightGBM_v2_df["hour_cos"] = np.cos(2 * np.pi * LightGBM_v2_df["hour"] / 24)
LightGBM_v2_df["dow_sin"] = np.sin(2 * np.pi * LightGBM_v2_df["dayofweek"] / 7)
LightGBM_v2_df["dow_cos"] = np.cos(2 * np.pi * LightGBM_v2_df["dayofweek"] / 7)

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

**hour_sin and hour_cos:** These features use sine and cosine transformations to encode the hour of the day in a cyclical manner. This helps the model understand that hours are part of a continuous cycle (24 hours in a day), avoiding the issue of treating 23:00 and 00:00 as completely different values.
dow_sin and dow_cos: Similarly, these features use sine and cosine transformations to encode the day of the week in a cyclical manner.

This helps the model understand that days of the week are part of a continuous cycle (7 days in a week), avoiding the issue of treating Sunday and Monday as completely different values.

<a id="LGBM2"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part V.2 Implementing the model

<a id="model"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part V.2.A Train/test split by shuffled 7-day blocks

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

To ensure a robust evaluation of our model while preserving the temporal structure of the data, we implement a block-based train-test split:


**Block Creation:** : We create blocks of data where each block represents a 7-day period. This is done by calculating the number of days since the minimum date in the index and dividing by 7. This ensures that each block contains a full week of data.



**Random Shuffling of Blocks:** We shuffle the unique block IDs randomly to distribute the blocks across the train and test sets. This randomness helps in reducing bias and ensures that the model is evaluated on a representative sample of the data.



**Train-Test Split:** We allocate 80% of the blocks to the training set and 20% of the blocks to the test set. This ensures that the temporal structure of the data is preserved within each set, which is crucial for time series analysis.
train_blocks: The blocks assigned to the training set.
test_blocks: The blocks assigned to the test set.



**Data Segregation:** We filter the original DataFrame to create separate training and testing DataFrames based on the block IDs.
train_df: The DataFrame containing the training data.
test_df: The DataFrame containing the test data.



This block-based approach ensures that the temporal dependencies within the data are respected, providing a more realistic evaluation of the model's performance on unseen data.


In [99]:
# Create a block index (1 block = 7 days)
LightGBM_v2_df["block_id"] = (
    (LightGBM_v2_df.index - LightGBM_v2_df.index.min()).days // 7
)

# Shuffle block IDs randomly
unique_blocks = LightGBM_v2_df["block_id"].unique()
np.random.seed(42)
np.random.shuffle(unique_blocks)

# Define 80% train, 20% test by block
n_train_blocks = int(len(unique_blocks) * 0.8)
train_blocks = unique_blocks[:n_train_blocks]
test_blocks = unique_blocks[n_train_blocks:]

train_df = LightGBM_v2_df[LightGBM_v2_df["block_id"].isin(train_blocks)]
test_df = LightGBM_v2_df[LightGBM_v2_df["block_id"].isin(test_blocks)]

# Drop helper column
train_df = train_df.drop(columns=["block_id"])
test_df = test_df.drop(columns=["block_id"])


<a id="split"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part V.2.B Splitting the Data

In [100]:
feature_cols = [
    c for c in LightGBM_v2_df.columns
    if c not in [target_col, "ww line 1", "ww line 2", "ww line 3", "ww line 4","block_id"]
]

X_train = train_df[feature_cols]
y_train = train_df[target_col]
X_test = test_df[feature_cols]
y_test = test_df[target_col]

<a id="split"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part V.2.C Finding the hyperparameters

In [101]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    """Objective function for Optuna hyperparameter optimization"""
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 800),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'verbose': -1,
        'random_state': 42
    }

    model = LGBMRegressor(**params)
    model.fit(X_train, y_train)
    
    preds = model.predict(X_val)
    mae = mean_absolute_error(y_val, preds)
    return mae

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=40, show_progress_bar=False)

best_params = study.best_params
print(" Best parameters found by Optuna:")
for k, v in best_params.items():
    print(f"  {k}: {v}")

<a id="split"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;">Part V.2.D Model and Evaluation

In [ ]:
best_model = LGBMRegressor(**best_params, random_state=42, verbose=-1)
best_model.fit(
    np.concatenate([X_train, X_val]),
    np.concatenate([y_train, y_val])
)

In [ ]:
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModel Evaluation on Test Set:")
print(f"MAE = {mae:.2f}")
print(f"MSE = {mse:.2f}")
print(f"R²  = {r2:.3f}")

In [ ]:
# plotting the prediction

plt.figure(figsize=(14, 5))
plt.plot(y_test.values[:500], label="Actual", linewidth=2)
plt.plot(y_pred[:500], label="Predicted", linewidth=2)
plt.title("Wastewater Volume — Predicted vs Actual (first 500 samples)")
plt.xlabel("Sample index")
plt.ylabel("Wastewater volume")
plt.legend()
plt.grid(True)
plt.show()

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

We can see here that the peaks are detected much more accurately by the model.
However, the MAE was lower with the previous model. This one is more “reactive” (or “sensitive”), meaning it tends to oscillate more around small fluctuations, which reduces its point-by-point precision.
But because it captures large variations and extreme values better, its overall R² score is higher.

Overall, I htink this model is likely more useful for real-world forecasting, as accurately anticipating major peaks is probably more valuable than achieving very high precision on minor fluctuations.

<a id="setup"></a>
# <p style="color:#E71B24; font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px; text-align:center;">PART VII - Testing a more advanced model using Deep Learning - Ts2Vec</p>

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">

In this final step, we will examine a more advanced model that directly addresses a challenge we identified earlier: the presence of multiple phenomena evolving across different temporal scales.  

Since the data contains both short-term fluctuations and long-term structural patterns, the key objective is to learn a **contextual, multi-scale temporal representation** of the features. This also naturally supports the **multivariate nature** of the problem (ww1 + ww2 + ww3 + ww4), allowing the model to encode how variables interact and evolve together over time.

What makes this approach particularly appealing is its ability to capture **complex, evolving dynamics** at several temporal resolutions simultaneously. The model is designed to encode both local variations and global trends, providing a richer and more expressive understanding of temporal behavior than traditional architectures.

The idea comes from this paper [https://arxiv.org/abs/2106.10466](https://arxiv.org/abs/2106.10466) — which provides both the theoretical foundations and a ready-to-use implementation. Given its relevance and the availability of the code, testing this model became a compelling choice for our experiments.

Moreover, the original paper describes not only how to build such representations, but also how to use them for robust forecasting tasks. Across many datasets, the proposed method achieves **state-of-the-art results**, demonstrating strong generalization and predictive power.


<a id="FE_LGBM"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part VII.1 Preparing the Data

In [158]:
ts2vec_df = df.copy()

In [159]:
# resample df into daily_df taking 24 entry and averaging them into one daily entry
ts2vec_df.index.name = 'date'               # give the index a name so reset_index yields 'date'
daily_df = ts2vec_df.resample('D').mean()
daily_df.reset_index(inplace=True)

In [ ]:
daily_df.info()

In [ ]:
#split train and test set with 2/3 and 1/3 of the data
n = len(ts2vec_df)
train_size = int(n * (2/3))
train_df = ts2vec_df[:train_size]
test_df = ts2vec_df[train_size:]

print(f"Train set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")

In [ ]:
print(train_df.columns)

In [12]:
# remove "wastewater volume" from train and test set
train_df = train_df.drop(columns=['wastewater volume'])
test_df = test_df.drop(columns=['wastewater volume'])


# add a dimesion to train and test set to make them 3D (samples, timesteps, features)
train_data = np.expand_dims(train_df.values, axis=0)
test_data = np.expand_dims(test_df.values, axis=0)


In [ ]:
print("train_data shape:", np.array(train_data).shape)
print("test_data shape:", np.array(test_data).shape)

<a id="FE_LGBM"></a>
## <p style="color: rgb(300, 110, 110); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part VII.2 Implementing the model 

<a id="new_dataframe"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part VII.2.A Training ts2vec

In [ ]:
# train TS2Vec model on train set


model = TS2Vec(
    input_dims=train_data.shape[2],
    device='cuda' if torch.cuda.is_available() else 'cpu',
    max_train_length=500
)


In [ ]:
loss_log = model.fit(
    train_data,
    verbose=True
)

In [16]:
train_repr = model.encode(train_data,causal=True)

In [17]:
test_repr = model.encode(test_data,causal=True)

In [ ]:
print(train_repr.shape)

<a id="new_dataframe"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part VII.2.B Linear regression model on the encoded train set

In [19]:
# we are going to train a simple linear regression model on the test set repr to predict the 4 ww lines

X_train = []
y_train = []

post_len=24

for i in range(0, train_repr.shape[1] - post_len,1):
    X_train.append(np.concatenate((train_repr[0, i].flatten(), train_df.iloc[i:i + post_len][["temperature_2m","rain","relative_humidity_2m"]].values.flatten())))
    y_train.append(train_df.iloc[i:i + post_len][['ww line 1', 'ww line 2', 'ww line 3', 'ww line 4']].values.flatten())

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)

<a id="new_dataframe"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part VII.2.C Evalution of the model

In [24]:
# evaluate on test set
X_test = []
y_test = []
for i in range(1, test_repr.shape[1] - post_len,25):
    X_test.append(np.concatenate((test_repr[0, i].flatten(), test_df.iloc[i:i + post_len][["temperature_2m","rain","relative_humidity_2m"]].values.flatten())))
    y_test.append(test_df.iloc[i:i + post_len][['ww line 1', 'ww line 2', 'ww line 3', 'ww line 4']].values.flatten())


In [ ]:
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {mse}")

In [ ]:
print("y_test shape:", np.array(y_test).shape)

<a id="new_dataframe"></a>
### <p style="color: rgb(250, 150, 150); font-family:'Futura', 'Century Gothic', 'Verdana', sans-serif; font-weight:600; letter-spacing:0.5px;"> Part VII.2.D Recreating the wastewater volume feature to plot the results

In [ ]:
# reacreate the feature wastewater (juste divide the 96 values in 4 clusters and sum them)
y_test_sum = np.sum(np.array(y_test).reshape(-1, 24, 4), axis=2)
y_pred_sum = np.sum(y_pred.reshape(-1, 24, 4), axis=2)

In [ ]:
#compute mse on the sum,R2score,MAE
mse_sum = mean_squared_error(y_test_sum, y_pred_sum)
r2 = r2_score(y_test_sum, y_pred_sum)
mae = mean_absolute_error(y_test_sum, y_pred_sum)
print(f"Test MSE on sum: {mse_sum}")
print(f"Test R2 score on sum: {r2}")
print(f"Test MAE on sum: {mae}")

In [ ]:
#shape of y_test_sum
print("y_test_sum shape:", y_test_sum.shape)

In [ ]:
#compare true vs predicted sum over test set
N=2
for i in range(N):
    plt.figure(figsize=(10, 4))
    plt.plot(y_test_sum[i], label='True Sum')
    plt.plot(y_pred_sum[i], label='Predicted Sum')
    plt.title(f'Comparison of True vs Predicted Wastewater Volume Sum - Sample {i+1}')
    plt.xlabel('Hour')
    plt.ylabel('Wastewater Volume Sum')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
y_pred = np.concatenate(y_pred_sum, axis=0)
y_test = np.concatenate(y_test_sum, axis=0)

N=250
i=50
plt.figure(figsize=(15, 5))
plt.plot(y_test[i:i+N], label='True Sum', alpha=0.7)
plt.plot(y_pred[i:i+N], label='Predicted Sum', alpha=0.7)
plt.title('Comparison of True vs Predicted Wastewater Volume Sum')
plt.xlabel('Hour')
plt.ylabel('Wastewater Volume Sum')
plt.legend()
plt.grid()
plt.show()

<div style="border-radius:10px; padding: 15px; background-color:rgb(36, 117, 179); font-size:120%; text-align:left">
In conclusion, this project allowed me to familiarize myself with an example of what an industrial prediction project at Veolia might look like. 

This dataset was very rich and therefore particularly interesting to explore and try to understand in depth. It also allowed me to do some research and discover new models. However, there are still several avenues that I did not have time to explore sufficiently, such as ARIMA statistical models, which are widely used in time series but which I found complicated to configure in order to understand a problem of this complexity (requiring a multi-scale temporal approach). 

Finally, in a project of this type, compromises may have to be made (such as choosing between a model that is more accurate for small variations or one that better detects peaks, see light GBM 1 and 2), and in this case, regular contact with the project's sponsoring teams would be particularly useful, which was not possible in this project.
